# [【SOTA】マイナビ × SIGNATE Student Cup 2019: 賃貸物件の家賃予測](https://signate.jp/competitions/264)

## 1. データ準備

### 1.1. データ読込

In [83]:
import pandas as pd
import numpy as np
import pathlib
import os

# 学習データ、テストデータの読み込み
train_path = pathlib.Path("./DATA/train.csv")
test_path = pathlib.Path("./DATA/test.csv")

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [84]:
train_data.head()

,id,賃料,所在地,アクセス,間取り,築年数,方角,面積,所在階,バス・トイレ,キッチン,放送・通信,室内設備,駐車場,周辺環境,建物構造,契約期間
0,1,75000,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,9年9ヶ月,南東,20.01m2,1階／12階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,RC（鉄筋コンクリート）,2年間
1,2,76000,東京都中央区月島３丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩9分\t\t日比谷線\...,1R,44年10ヶ月,NaN,16.5m2,5階／10階建,専用トイレ／\tシャワー／\t温水洗浄便座,ガスコンロ／\tシステムキッチン\t／\t給湯,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【スーパー】 1283m,鉄骨造,2年間
2,3,110000,東京都渋谷区笹塚２丁目,京王線\t笹塚駅\t徒歩6分\t\t京王線\t代田橋駅\t徒歩7分\t\t京王線\t明大前駅...,1K,8年6ヶ月,南,22.05m2,12階／15階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\t光ファイバー／\tCSアンテナ／\tBSアンテナ,エアコン付\tウォークインクローゼット\tシューズボックス／\tバルコニー／\tフローリング...,"駐輪場\t空有\tバイク置き場\t空有\t駐車場\t近隣\t30,000円(税込)\t距離100m",【スーパー】 89m\t【コンビニ】 184m\t【コンビニ】 392m\t【スーパー】 492m,RC（鉄筋コンクリート）,2年間
3,4,150000,東京都杉並区高円寺南２丁目23-2,総武線・中央線（各停）\t高円寺駅\t徒歩9分\t\t丸ノ内線(池袋－荻窪)\t新高円寺駅\...,2LDK,29年4ヶ月,南,60.48m2,3階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t温水洗浄便座／\t洗面台独立,ガスコンロ／\t給湯,インターネット対応／\t光ファイバー,エアコン付\tシューズボックス／\tバルコニー／\t2面採光／\t室内洗濯機置場／\tエレベ...,駐車場\t無\t駐輪場\t無\tバイク置き場\t無,【スーパー】 225m\t【スーパー】 448m\t【スーパー】 619m\t【スーパー】 ...,RC（鉄筋コンクリート）,2年間\t※この物件は\t定期借家\tです。
4,5,74000,東京都葛飾区金町３丁目7-2,京成金町線\t京成金町駅\t徒歩5分\t\t常磐線\t金町(東京都)駅\t徒歩7分\t\t京...,2DK,31年7ヶ月,南,39.66m2,1階／2階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能,給湯／\t独立キッチン,NaN,バルコニー／\tフローリング／\t室内洗濯機置場\t公営水道／\t下水,"駐車場\t近隣\t17,000円(税込)\t距離300m\t駐輪場\t無\tバイク置き場\t無",【スーパー】 193m\t【スーパー】 298m\t【スーパー】 660m\t【スーパー】 ...,木造,2年間


## 2. 前処理

### 2.1. データ選択

In [85]:
train_data_1 = train_data[["id", "所在地", "賃料", "間取り", "築年数","方角", "面積", "所在階", "建物構造"]]
test_data_1 = test_data[["id", "所在地", "間取り", "築年数","方角",  "面積", "所在階", "建物構造"]]

### 2.2. 間取りの数値化

In [86]:
# 16m2で11R扱いの物件を1Rに修正する
# test_data.loc[test_data["間取り"]=="11R"]
test_data_1 = test_data_1.replace("11R", "1R")

In [87]:
# 間取りにlabel encodingを適用
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(pd.concat([train_data_1["間取り"],test_data_1["間取り"]]))

train_data_1["間取りID"] = le.transform(train_data_1["間取り"])
test_data_1["間取りID"] = le.transform(test_data_1["間取り"])

<ipython-input-87-95a8caebc059>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_1["間取りID"] = le.transform(train_data_1["間取り"])


### 2.3. 面積の数値化

In [88]:
# 面積のカラムタイトルを面積[m2]に置換
train_data_1 = train_data_1.rename(columns={"面積": "面積m2"})
test_data_1 = test_data_1.rename(columns={"面積": "面積m2"})

In [89]:
# カラムタイトル面積[m2]の要素からm2を削除
train_data_1['面積m2'] = train_data_1['面積m2'].str.replace('m2', '').astype(float)
test_data_1['面積m2'] = test_data_1['面積m2'].str.replace('m2', '').astype(float)

### 2.4. 所在階の数値化

In [90]:
# 所在階が総階数よりも大きい場合、値を入れ替える
train_data_1 = train_data_1.replace("2階／1階建", "1階／2階建")
train_data_1 = train_data_1.replace("4階／2階建", "2階／4階建")
train_data_1 = train_data_1.replace("7階／2階建", "2階／7階建")
train_data_1 = train_data_1.replace("10階／8階建", "8階／10階建")

test_data_1 = test_data_1.replace("2階／1階建", "1階／2階建")
test_data_1 = test_data_1.replace("3階／2階建", "2階／3階建")
test_data_1 = test_data_1.replace("4階／1階建", "1階／4階建")
test_data_1 = test_data_1.replace("4階／2階建", "2階／4階建")
test_data_1 = test_data_1.replace("4階／3階建（地下1階）", "3階／4階建（地下1階）")
test_data_1 = test_data_1.replace("8階／6階建", "6階／8階建")
test_data_1 = test_data_1.replace("13階／4階建（地下1階）", "4階／13階建（地下1階）")
test_data_1 = test_data_1.replace("21階／3階建", "3階／21階建")
test_data_1 = test_data_1.replace("44階／4階建", "4階／44階建")

In [91]:
# 所在階の中身を"／"で2つの列に分割
train_data_1 = pd.concat([train_data_1, train_data_1['所在階'].str.split('／', expand=True)], axis=1)
test_data_1 = pd.concat([test_data_1, test_data_1['所在階'].str.split('／', expand=True)], axis=1)

In [92]:
# 分割した所在階のカラム名変更　0:所在、1:階層
train_data_1 = train_data_1.rename(columns={0:"所在", 1:"階層"})
test_data_1 = test_data_1.rename(columns={0:"所在", 1:"階層"})

In [93]:
# 所在と階層の要素を整形
train_data_1['所在'] = train_data_1['所在'].str.replace('階', '')
train_data_1['所在'] = train_data_1['所在'].str.replace('建', '')
train_data_1['所在'] = train_data_1['所在'].str.replace('地下', '-')
train_data_1['所在'] = train_data_1['所在'].str.replace('\（.*\）', '', regex=True)
train_data_1['階層'] = train_data_1['階層'].str.replace('階建', '')
train_data_1['階層'] = train_data_1['階層'].str.replace('\（.*\）', '', regex=True)
train_data_1['所在'] = train_data_1['所在'].replace('', np.nan)

test_data_1['所在'] = test_data_1['所在'].str.replace('階', '')
test_data_1['所在'] = test_data_1['所在'].str.replace('建', '')
test_data_1['所在'] = test_data_1['所在'].str.replace('地下', '-')
test_data_1['所在'] = test_data_1['所在'].str.replace('\（.*\）', '', regex=True)
test_data_1['階層'] = test_data_1['階層'].str.replace('階建', '')
test_data_1['階層'] = test_data_1['階層'].str.replace('\（.*\）', '', regex=True)
test_data_1['所在'] = test_data_1['所在'].replace('', np.nan)

In [94]:
# # "階層"がNoneの箇所を"所在"の値で埋める
# train_data_1['階層'].fillna(train_data_1['所在'], inplace=True)
# test_data_1['階層'].fillna(test_data_1['所在'], inplace=True)

In [95]:
# 所在と階層の要素をfloat型に変換する
train_data_1['所在'] = train_data_1['所在'].astype(float)
train_data_1['階層'] = train_data_1['階層'].astype(float)

test_data_1['所在'] = test_data_1['所在'].astype(float)
test_data_1['階層'] = test_data_1['階層'].astype(float)

In [96]:
# 所在階のカラムを削除
train_data_1 = train_data_1.drop('所在階', axis=1)
test_data_1 = test_data_1.drop('所在階', axis=1)

In [97]:
# # 所在のNaNを埋める
# train_data_1.loc[train_data_1["所在"].isnull(), "所在"] = train_data_1["階層"]
# test_data_1.loc[test_data_1["所在"].isnull(), "所在"] = test_data_1["階層"]

In [98]:
# # 所在も階層も空欄のデータは間取りと面積の近いデータで埋める
# print(test_data_1[(test_data_1["間取りID"] == 21) & (test_data_1["面積m2"] > 90) & (test_data_1["面積m2"] < 95)].mean())
# test_data_1.loc[test_data_1["id"]==40675, "所在"] = float(6)
# test_data_1.loc[test_data_1["id"]==40675, "階層"] = float(9)

### 2.5. 築年数の数値化

In [99]:
# 新築の場合は全て0にする
train_data_1.loc[train_data_1["築年数"]=="新築", "築年数"] = float(0)
test_data_1.loc[test_data_1["築年数"]=="新築", "築年数"] = float(0)

In [100]:
# 築年数を数値に変換する
train_data_1 = pd.concat([train_data_1, train_data_1['築年数'].str.split('年', expand=True)], axis=1)
test_data_1 = pd.concat([test_data_1, test_data_1['築年数'].str.split('年', expand=True)], axis=1)

In [101]:
# ヶ月を消す
train_data_1[1] = train_data_1[1].str.replace('ヶ月', '')
test_data_1[1] = test_data_1[1].str.replace('ヶ月', '')

In [102]:
# 築年数をfloat変換
train_data_1["築年数"] = train_data_1[0].astype(float) + (train_data_1[1].astype(float) / float(12))
test_data_1["築年数"] = test_data_1[0].astype(float) + (test_data_1[1].astype(float) / float(12))

In [103]:
test_data_1.head()

,id,所在地,間取り,築年数,方角,面積m2,建物構造,間取りID,所在,階層,0,1
0,31471,東京都世田谷区深沢５丁目1-27,2LDK,49.000000,南,50.22,鉄骨造,13,8.0,8.0,49,0
1,31472,東京都目黒区八雲１丁目11-8,1R,0.166667,南東,20.88,RC（鉄筋コンクリート）,8,3.0,4.0,0,2
2,31473,東京都豊島区池袋本町２丁目22-2,1K,23.333333,南東,26.93,鉄骨造,2,1.0,4.0,23,4
3,31474,東京都杉並区和泉１丁目,1K,36.166667,南東,23.57,木造,2,1.0,2.0,36,2
4,31475,東京都杉並区堀ノ内２丁目,2LDK,45.333333,南,50.00,RC（鉄筋コンクリート）,13,4.0,4.0,45,4


In [104]:
print(test_data_1[test_data_1["築年数"].isnull()])

          id               所在地       間取り  築年数  方角   面積m2          建物構造  間取りID  \
77     31548      東京都大田区東矢口２丁目  1K+S(納戸)  NaN   南  23.80           鉄骨造      3   
108    31579  東京都世田谷区上馬２丁目26-8        1K  NaN  南西  26.44  RC（鉄筋コンクリート）      2   
112    31583       東京都台東区台東２丁目      1LDK  NaN   北  41.65           鉄骨造      4   
156    31627  東京都品川区南大井４丁目20-5        1R  NaN  南東  20.56  RC（鉄筋コンクリート）      8   
162    31633     東京都江東区扇橋２丁目23        1K  NaN   西  25.34  RC（鉄筋コンクリート）      2   
...      ...               ...       ...  ...  ..    ...           ...    ...   
31141  62612  東京都世田谷区桜丘４丁目15-9      2LDK  NaN  南西  60.81            木造     13   
31176  62647   東京都江戸川区中葛西８丁目22      1LDK  NaN   東  40.06  RC（鉄筋コンクリート）      4   
31181  62652       東京都新宿区原町３丁目      1LDK  NaN  南東  40.26           鉄骨造      4   
31182  62653       東京都新宿区原町３丁目      2LDK  NaN  南西  54.75           鉄骨造     13   
31212  62683       東京都新宿区原町３丁目      1LDK  NaN  北西  40.79           鉄骨造      4   

        所在    階層    0    1 

In [105]:
# 0と1の列を消す
train_data_1 = train_data_1.drop(0, axis=1)
train_data_1 = train_data_1.drop(1, axis=1)

test_data_1 = test_data_1.drop(0, axis=1)
test_data_1 = test_data_1.drop(1, axis=1)

### 2.6. 緯度経度情報の追加

#### [このサイト](https://ktgis.net/gcode/geocoding.html)を利用する

In [106]:
# IDと住所をファイル出力
train_coordinate = train_data[["id", "所在地"]]
test_coordinate = test_data[["id", "所在地"]]

train_coordinate.to_csv("train_coordinate.csv", header=False, index=False)
test_coordinate.to_csv("test_coordinate.csv", header=False, index=False)

In [107]:
# 緯度、経度情報の読み込み(id, 所在地, 経度, 緯度)
train_coordinate_addvalue = pd.read_excel("train_coordinate_addvalue.xlsx")
test_coordinate_addvalue = pd.read_excel("test_coordinate_addvalue.xlsx")

In [108]:
train_data_1 = pd.merge(train_data_1, train_coordinate_addvalue[["id", "経度", "緯度"]], on='id')
test_data_1 = pd.merge(test_data_1, test_coordinate_addvalue[["id", "経度", "緯度"]], on='id')

### 2.7. 部屋数の追加

In [109]:
# 部屋数のマージ
number_of_rooms = pd.read_excel("number_of_rooms.xlsx")
train_data_1 = pd.merge(train_data_1, number_of_rooms[["間取り", "部屋数"]], on='間取り')
test_data_1 = pd.merge(test_data_1, number_of_rooms[["間取り", "部屋数"]], on='間取り')

In [110]:
# インデックスの振り直し
train_data_1 = train_data_1.sort_values("id")
train_data_1 = train_data_1.reset_index(drop=True)

test_data_1 = test_data_1.sort_values("id")
test_data_1 = test_data_1.reset_index(drop=True)

### 2.8. 1部屋当たりの面積追加

In [111]:
# 面積と部屋数から1部屋当たりの面積を算出
train_data_1["1部屋当たり面積m2"] = (train_data_1["面積m2"] / train_data_1["部屋数"]).astype(float)
test_data_1["1部屋当たり面積m2"] = (test_data_1["面積m2"] / test_data_1["部屋数"]).astype(float)

In [112]:
# 間取りと間取りIDは削除する
train_data_1 = train_data_1.drop('間取り', axis=1)
test_data_1 = test_data_1.drop('間取り', axis=1)

# train_data_1 = train_data_1.drop('間取りID', axis=1)
# test_data_1 = test_data_1.drop('間取りID', axis=1)

#### 2.9. 建物の高さ率を追加(高さ率=所在/階層)

In [113]:
# 高さ率の算出
train_data_1["高さ率"] = (train_data_1["所在"] / train_data_1["階層"]).astype(float)
test_data_1["高さ率"] = (test_data_1["所在"] / test_data_1["階層"]).astype(float)

### 2.10. 建物構造の数値化

In [114]:
train_data_1["建物構造"].value_counts()

RC（鉄筋コンクリート）                   17505
木造                              5512
鉄骨造                             3726
SRC（鉄骨鉄筋コンクリート）                 2429
軽量鉄骨                            1812
ALC（軽量気泡コンクリート）                  297
その他                              136
PC（プレキャスト・コンクリート（鉄筋コンクリート））       46
HPC（プレキャスト・コンクリート（重量鉄骨））           4
ブロック                               3
Name: 建物構造, dtype: int64

In [115]:
# 建物構造にlabel encodingを適用
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(pd.concat([train_data_1["建物構造"],test_data_1["建物構造"]]))

train_data_1["建物構造ID"] = le.transform(train_data_1["建物構造"])
test_data_1["建物構造ID"] = le.transform(test_data_1["建物構造"])

In [116]:
test_data_1.head()

,id,所在地,築年数,方角,面積m2,建物構造,間取りID,所在,階層,経度,緯度,部屋数,1部屋当たり面積m2,高さ率,建物構造ID
0,31471,東京都世田谷区深沢５丁目1-27,49.000000,南,50.22,鉄骨造,13,8.0,8.0,139.652762,35.617708,3.5,14.348571,1.00,10
1,31472,東京都目黒区八雲１丁目11-8,0.166667,南東,20.88,RC（鉄筋コンクリート）,8,3.0,4.0,139.672871,35.617709,1.0,20.880000,0.75,3
2,31473,東京都豊島区池袋本町２丁目22-2,23.333333,南東,26.93,鉄骨造,2,1.0,4.0,139.709049,35.741996,1.0,26.930000,0.25,10
3,31474,東京都杉並区和泉１丁目,36.166667,南東,23.57,木造,2,1.0,2.0,139.658270,35.673602,1.0,23.570000,0.50,7
4,31475,東京都杉並区堀ノ内２丁目,45.333333,南,50.00,RC（鉄筋コンクリート）,13,4.0,4.0,139.650468,35.689411,3.5,14.285714,1.00,3


In [117]:
# 不要になった建物構造カラムを削除する
train_data_1 = train_data_1.drop('建物構造', axis=1)
test_data_1 = test_data_1.drop('建物構造', axis=1)

### 2.11. 新規カラムに「賃料/面積」を追加

In [118]:
train_data_1["賃料/面積"] = train_data_1["賃料"] / train_data_1["面積m2"]

In [119]:
train_data_1.head()

,id,所在地,賃料,築年数,方角,面積m2,間取りID,所在,階層,経度,緯度,部屋数,1部屋当たり面積m2,高さ率,建物構造ID,賃料/面積
0,1,東京都北区滝野川３丁目,75000,9.750000,南東,20.01,2,1.0,12.0,139.727873,35.747759,1.0,20.01,0.083333,3,3748.125937
1,2,東京都中央区月島３丁目,76000,44.833333,NaN,16.50,8,5.0,10.0,139.780485,35.662406,1.0,16.50,0.500000,10,4606.060606
2,3,東京都渋谷区笹塚２丁目,110000,8.500000,南,22.05,2,12.0,15.0,139.667375,35.675358,1.0,22.05,0.800000,3,4988.662132
3,4,東京都杉並区高円寺南２丁目23-2,150000,29.333333,南,60.48,13,3.0,4.0,139.648495,35.700193,3.5,17.28,0.750000,3,2480.158730
4,5,東京都葛飾区金町３丁目7-2,74000,31.583333,南,39.66,9,1.0,2.0,139.872315,35.766413,3.0,13.22,0.500000,7,1865.859808


### 2.12. 東京23区の地域ごとに、住宅地の平均地価情報を追加

In [120]:
# 区の情報の確認(23区だけだった)
sample = train_data["所在地"].str.extract("東京都(.+区)", expand=True)
sample.value_counts()

世田谷区    3023
足立区     2191
大田区     2000
杉並区     1769
練馬区     1752
江戸川区    1702
板橋区     1624
港区      1513
江東区     1508
中央区     1279
品川区     1272
豊島区     1271
新宿区     1223
葛飾区     1212
中野区     1211
北区      1045
目黒区     1023
墨田区     1010
渋谷区      985
台東区      932
文京区      892
荒川区      576
千代田区     457
dtype: int64

In [121]:
print(len(train_data_1))
print(len(test_data_1))

31470
31262


In [122]:
# 訓練データとテストデータに、エリア情報を追加
train_data_1["エリア"] = ""
test_data_1["エリア"] = ""
train_data_1["エリア"] = train_data_1["所在地"].str.extract("東京都((.+区.+?)[０-９]|(.+区.+?)[0-9]|(.+区.+))", expand=True)
test_data_1["エリア"] = test_data_1["所在地"].str.extract("東京都((.+区.+?)[０-９]|(.+区.+?)[0-9]|(.+区.+))", expand=True)

In [123]:
print(len(train_data_1))
print(len(test_data_1))

31470
31262


In [124]:
# 一部エリア情報に数値が残ってしまうので、消す
train_data_1["エリア"] = train_data_1["エリア"].str.replace("[０-９]|[0-9]", "")
train_data_1["エリア"] = train_data_1["エリア"].str.replace("一丁目", "")
train_data_1["エリア"] = train_data_1["エリア"].str.replace("-", "")

test_data_1["エリア"] = test_data_1["エリア"].str.replace("[０-９]|[0-9]", "")
test_data_1["エリア"] = test_data_1["エリア"].str.replace("一丁目", "")
test_data_1["エリア"] = test_data_1["エリア"].str.replace("二丁目", "")
test_data_1["エリア"] = test_data_1["エリア"].str.replace("四丁目", "")

In [125]:
print(len(train_data_1))
print(len(test_data_1))

31470
31262


In [126]:
# 坪単価情報の読込～結合
wards_value = pd.read_excel("23区_地域毎_坪単価.xlsx")
train_data_1 = pd.merge(train_data_1, wards_value, on="エリア", how="left")
test_data_1 = pd.merge(test_data_1, wards_value, on="エリア", how="left")
train_data_1 = train_data_1.sort_values("id")
test_data_1 = test_data_1.sort_values("id")

In [127]:
# 地価の抜けデータを平均値で補間
# 江東区
train_data_1.loc[train_data_1["id"]==2767, "平均坪単価（万円）"] = 206.6
train_data_1.loc[train_data_1["id"]==7559, "平均坪単価（万円）"] = 206.6
# 港区
train_data_1.loc[train_data_1["id"]==17654, "平均坪単価（万円）"] = 1321.4

In [128]:
print(len(train_data_1))
print(len(test_data_1))

31470
31262


In [129]:
print(train_data_1[train_data_1["平均坪単価（万円）"].isnull()])
print(test_data_1[test_data_1["平均坪単価（万円）"].isnull()])

Empty DataFrame
Columns: [id, 所在地, 賃料, 築年数, 方角, 面積m2, 間取りID, 所在, 階層, 経度, 緯度, 部屋数, 1部屋当たり面積m2, 高さ率, 建物構造ID, 賃料/面積, エリア, 区, 地名, 平均坪単価（万円）]
Index: []
Empty DataFrame
Columns: [id, 所在地, 築年数, 方角, 面積m2, 間取りID, 所在, 階層, 経度, 緯度, 部屋数, 1部屋当たり面積m2, 高さ率, 建物構造ID, エリア, 区, 地名, 平均坪単価（万円）]
Index: []


In [130]:
# 不要なカラムを削除する
train_data_1 = train_data_1.drop(["所在地", "エリア", "区", "地名"], axis=1)
test_data_1 = test_data_1.drop(["所在地", "エリア", "区", "地名"], axis=1)

In [131]:
train_data_1.head()

,id,賃料,築年数,方角,面積m2,間取りID,所在,階層,経度,緯度,部屋数,1部屋当たり面積m2,高さ率,建物構造ID,賃料/面積,平均坪単価（万円）
0,1,75000,9.750000,南東,20.01,2,1.0,12.0,139.727873,35.747759,1.0,20.01,0.083333,3,3748.125937,180.0
1,2,76000,44.833333,NaN,16.50,8,5.0,10.0,139.780485,35.662406,1.0,16.50,0.500000,10,4606.060606,323.0
2,3,110000,8.500000,南,22.05,2,12.0,15.0,139.667375,35.675358,1.0,22.05,0.800000,3,4988.662132,324.0
3,4,150000,29.333333,南,60.48,13,3.0,4.0,139.648495,35.700193,3.5,17.28,0.750000,3,2480.158730,253.0
4,5,74000,31.583333,南,39.66,9,1.0,2.0,139.872315,35.766413,3.0,13.22,0.500000,7,1865.859808,111.0


### 2.13. 最寄り駅までの徒歩分数を追加

In [132]:
# 精度悪化の要因になるため無効化
# train_data_1["アクセス"] = (train_data["アクセス"].str.extract("徒歩(.+?)", expand=True)).astype(float)
# test_data_1["アクセス"] = (test_data["アクセス"].str.extract("徒歩(.+?)", expand=True)).astype(float)

### 2.14. 方角の数値化

In [133]:
train_data_1["方角"].fillna("missing", inplace=True)
test_data_1["方角"].fillna("missing", inplace=True)

In [134]:
# 方角にlabel encodingを適用
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(pd.concat([train_data_1["方角"],test_data_1["方角"]]))

train_data_1["方角ID"] = le.transform(train_data_1["方角"])
test_data_1["方角ID"] = le.transform(test_data_1["方角"])

In [150]:
# 方角のカラムを削除
train_data_1 = train_data_1.drop("方角", axis=1)
test_data_1 = test_data_1.drop("方角", axis=1)

### 2.15. 重複物件の抽出・削除

In [137]:
# 重複確認用のデータ出力
# train_duplicate = pd.concat([train_data[["id", "賃料", "所在地"]], train_data_1[["間取りID", "面積m2", "所在", "階層"]]], axis=1)
# test_duplicate = pd.concat([test_data[["id", "所在地"]], test_data_1[["間取りID", "面積m2", "所在", "階層"]]], axis=1)

In [138]:
# 結果の出力
# train_duplicate.to_excel("train_duplicate.xlsx", header=True, index=False)
# test_duplicate.to_excel("test_duplicate.xlsx", header=True, index=False)

In [139]:
# 重複対象のデータ読み込み
train_duplicate_flag = pd.read_excel("train_duplicate_flag.xlsx")
test_duplicate_flag = pd.read_excel("test_duplicate_flag.xlsx")

In [140]:
# データ数のチェック
print("Train:", len(train_data_1), "-", len(train_duplicate_flag), "=", len(train_data_1)-len(train_duplicate_flag))
print("Test:", len(test_data_1), "-", len(test_duplicate_flag), "=", len(test_data_1)-len(test_duplicate_flag))

Train: 31470 - 31470 = 0
Test: 31262 - 31262 = 0


In [141]:
# 重複フラグを結合
train_data_1 = pd.merge(train_data_1, train_duplicate_flag, on="id")
train_data_1 = train_data_1.rename(columns={"賃料_x":"賃料"})
train_data_1 = train_data_1.drop('賃料_y', axis=1)

test_data_1 = pd.merge(test_data_1, test_duplicate_flag, on="id")

In [142]:
# testデータからのみ、flag=1.0の列を削除
print("【削除前】", "train:", len(train_data_1), ", ", "test:", len(test_data_1))
#train_data_1 = train_data_1.loc[train_data_1["flag"] != 1.0]
test_data_1 = test_data_1.loc[test_data_1["flag"] != 1.0]
print("【削除後】", "train:", len(train_data_1), ", ", "test:", len(test_data_1))

【削除前】 train: 31470 ,  test: 31262
【削除後】 train: 31470 ,  test: 24812


## 3. データチェック

In [143]:
train_data_1.isnull().sum()

id                0
賃料                0
築年数             889
方角                0
面積m2              0
間取りID             0
所在              120
階層              594
経度                0
緯度                0
部屋数               0
1部屋当たり面積m2        0
高さ率             714
建物構造ID            0
賃料/面積             0
平均坪単価（万円）         0
方角ID              0
flag          25975
dtype: int64

In [144]:
test_data_1.isnull().sum()

id                0
築年数             675
方角                0
面積m2              0
間取りID             0
所在              102
階層              476
経度                0
緯度                0
部屋数               0
1部屋当たり面積m2        0
高さ率             577
建物構造ID            0
平均坪単価（万円）         0
方角ID              0
flag          24812
dtype: int64

In [145]:
len(train_data_1)

31470

In [146]:
len(train_data)

31470

In [147]:
len(test_data_1)

24812

In [148]:
len(test_data)

31262

In [151]:
train_data_1.head()

,id,賃料,築年数,面積m2,間取りID,所在,階層,経度,緯度,部屋数,1部屋当たり面積m2,高さ率,建物構造ID,賃料/面積,平均坪単価（万円）,方角ID,flag
0,1,75000,9.750000,20.01,2,1.0,12.0,139.727873,35.747759,1.0,20.01,0.083333,3,3748.125937,180.0,5,NaN
1,2,76000,44.833333,16.50,8,5.0,10.0,139.780485,35.662406,1.0,16.50,0.500000,10,4606.060606,323.0,0,1.0
2,3,110000,8.500000,22.05,2,12.0,15.0,139.667375,35.675358,1.0,22.05,0.800000,3,4988.662132,324.0,4,NaN
3,4,150000,29.333333,60.48,13,3.0,4.0,139.648495,35.700193,3.5,17.28,0.750000,3,2480.158730,253.0,4,NaN
4,5,74000,31.583333,39.66,9,1.0,2.0,139.872315,35.766413,3.0,13.22,0.500000,7,1865.859808,111.0,4,NaN


In [152]:
test_data_1.head()

,id,築年数,面積m2,間取りID,所在,階層,経度,緯度,部屋数,1部屋当たり面積m2,高さ率,建物構造ID,平均坪単価（万円）,方角ID,flag
0,31471,49.000000,50.22,13,8.0,8.0,139.652762,35.617708,3.5,14.348571,1.00,10,208.0,4,NaN
1,31472,0.166667,20.88,8,3.0,4.0,139.672871,35.617709,1.0,20.880000,0.75,3,273.0,5,NaN
2,31473,23.333333,26.93,2,1.0,4.0,139.709049,35.741996,1.0,26.930000,0.25,10,186.0,5,NaN
3,31474,36.166667,23.57,2,1.0,2.0,139.658270,35.673602,1.0,23.570000,0.50,7,184.0,5,NaN
5,31476,25.500000,21.45,8,1.0,2.0,139.686023,35.691395,1.0,21.450000,0.50,7,294.0,8,NaN


## 4.学習 

In [153]:
# IDとflagの削除
train_data_1_no_ID = train_data_1.drop(["id", "flag"], axis=1)
test_data_1_no_ID = test_data_1.drop(["id", "flag"], axis=1)

In [154]:
# 特徴データと目的変数の設定
train_x = train_data_1_no_ID.drop(["賃料", "賃料/面積"], axis=1)
train_y = train_data_1_no_ID["賃料/面積"]

test_x = test_data_1_no_ID

In [155]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

scores = []
# GBDT用のハイパーパラメータ
params = {"objective": "reg:squarederror", "sileng":1, "random_state":71}
num_round = 50

# クロスバリデーション
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx, in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    # GBDTで学習実行
    dtrain = xgb.DMatrix(tr_x, label=tr_y) # enable_categorical=True
    dvalid = xgb.DMatrix(va_x, label=va_y)
    dtest = xgb.DMatrix(test_x)
    
    watchlist = [(dtrain, "train"), (dvalid, "eval")]
    model = xgb.train(params, dtrain, num_round, evals=watchlist)

[16:32:48] WARNING: ..\src\learner.cc:541: 
Parameters: { sileng } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:2640.36816	eval-rmse:2611.49194
[1]	train-rmse:1896.67810	eval-rmse:1863.53442
[2]	train-rmse:1388.44690	eval-rmse:1349.13806
[3]	train-rmse:1046.06323	eval-rmse:999.88324
[4]	train-rmse:821.59637	eval-rmse:768.70276
[5]	train-rmse:680.28516	eval-rmse:620.03302
[6]	train-rmse:586.77325	eval-rmse:531.34381
[7]	train-rmse:533.80347	eval-rmse:476.12622
[8]	train-rmse:503.33902	eval-rmse:443.98730
[9]	train-rmse:484.06250	eval-rmse:426.16788
[10]	train-rmse:473.29321	eval-rmse:416.13342
[11]	train-rmse:464.70474	eval-rmse:408.06131
[12]	train-rmse:443.69592	eval-rmse:402.73532
[13]	train-rmse:439.38007	eval-rmse:401.11755
[14]	train-rmse:432.92844	eval-rm

## 5. 予測

In [156]:
# テストデータでの予測
pred = model.predict(dtest)

In [157]:
pred

array([2772.2922, 5489.0034, 2947.1304, ..., 4556.7534, 3772.5093,
       3744.1917], dtype=float32)

In [158]:
len(pred)

24812

In [159]:
len(test_data_1)

24812

## 6. データ出力

In [160]:
# 予測結果の結合
test_data_1["賃料/面積"] = pred

In [161]:
# 予測結果から賃料を算出し、int型に変換
test_data_1["賃料"] = (test_data_1["面積m2"] * test_data_1["賃料/面積"]).astype(int)

In [162]:
result = test_data_1[["id", "賃料"]]

In [163]:
result

,id,賃料
0,31471,139224
1,31472,114610
2,31473,79366
3,31474,59593
5,31476,69333
...,...,...
31255,62726,100395
31257,62728,104332
31258,62729,103483
31259,62730,172630


In [164]:
len(result)

24812

In [165]:
# 除外していた重複idの賃料を戻す
test_duplicate_rentvalue = pd.read_excel("test_duplicate_rentvalue.xlsx")
result = pd.concat([result, test_duplicate_rentvalue])
result = result.sort_values("id")
len(result)

31262

In [166]:
result

,id,賃料
0,31471,139224
1,31472,114610
2,31473,79366
3,31474,59593
0,31475,90000
...,...,...
31257,62728,104332
31258,62729,103483
31259,62730,172630
6449,62731,280000


In [167]:
result.to_csv("result.csv", header=False, index=False)